# Data Parallelism

## Device
기본적으로 한 개의 GPU를 사용
```
device = torch.device("cuda:0")

model.to(device) # 모델 cuda로

mytensor = mytensor.to(device) # 모든 tensor cuda로
```

## DataParallel
여러 대의 GPU 사용
```
model = nn.DataParallel(model)
```


## Imports and parameters

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
# Parameters and DataLoaders
input_size = 5
output_size = 2

batch_size = 30
data_size = 100 

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Make Dummy Dataset

In [5]:
class RandomDataset(Dataset):
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)
        
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len

- Dataset : dataset 만들어주는 모듈
- DataLoader : dataset를 iterate로 만드는 모듈

In [6]:
rand_loader = DataLoader(dataset = RandomDataset(input_size, 100), batch_size=batch_size, shuffle=True)

## Simple Model

In [7]:
class Model(nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, input):
        output = self.fc(input)
        
        print("\tIn Model: input size", input.size(),
              "output size", output.size())
        
        return output

## Create Model and DataParallel

In [8]:
model = Model(input_size, output_size)

if torch.cuda.device_count() >1:
    print("Lets use", torch.cuda.device_count(), "GPU")
    model = nn.DataParallel(model)
    
model.to(device)

C:\Users\magenta_jy\Anaconda3\lib\site-packages\torch\cuda\__init__.py:116: UserWarning: 
    Found GPU0 GeForce 710M which is of cuda capability 2.1.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

## Run the Model

In [9]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outsize : input size", input.size(), "output_size", output.size())

RuntimeError: cuda runtime error (8) : invalid device function at C:/ProgramData/Miniconda3/conda-bld/pytorch_1524543037166/work/aten/src/THC/THCTensorCopy.cu:204

- batch size 별로 나누어서 처리한다.